In [1]:
from torch.utils.data import DataLoader
from concept_model.dataset import CUBImageToAttributes

training_data = CUBImageToAttributes(train=True)
test_data = CUBImageToAttributes(train=False)

batch_size = 16
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([16, 3, 299, 299])
Shape of y: torch.Size([16, 312]) torch.int64


In [2]:
import torch

model = torch.hub.load(
    "pytorch/vision:v0.10.0", "inception_v3", pretrained=False, num_classes=312
)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
model.to(device)

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(torch.float).to(device)
        # Compute prediction and loss
        logits, aux_logits = model(X)
        loss = loss_fn(logits, y) + 0.4 * loss_fn(aux_logits, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    correct_using_attributes = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y.to(torch.float)).item()
            correct += (
                ((torch.sigmoid(pred) > 0.5).to(torch.int64) == y)
                .type(torch.float)
                .all(dim=1)
                .sum()
                .item()
            )
            correct_attributes = torch.sum(
                ((torch.nn.Sigmoid()(pred) > 0.5).to(torch.int64) == y).to(torch.float)
            )
            num_attributes = y.shape[1]
            correct_using_attributes += correct_attributes.item() / num_attributes
    test_loss /= num_batches
    correct /= size
    correct_using_attributes /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Accuracy (Attribute-wise): {(100*correct_using_attributes):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )
    return correct_using_attributes


Using cache found in /home/joanna/.cache/torch/hub/pytorch_vision_v0.10.0
/home/joanna/interactive-concept-bottleneck/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/joanna/interactive-concept-bottleneck/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/joanna/interactive-concept-bottleneck/.venv/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), pl

Using cuda device


In [3]:
epochs = 500
max_acc = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    curr_acc = test(test_dataloader, model, loss_fn)
    if curr_acc > max_acc:
        torch.save(model.state_dict(), "image_to_attributes.pth")
        max_acc = curr_acc
print("Done!")


Epoch 1
-------------------------------
loss: 1.469870  [    0/ 5994]
loss: 1.400770  [ 1600/ 5994]
loss: 1.372612  [ 3200/ 5994]
loss: 1.409056  [ 4800/ 5994]
Test Error: 
 Accuracy: 0.0%, Accuracy (Attribute-wise): 56.3%, Avg loss: 0.935561 

Epoch 2
-------------------------------
loss: 1.405358  [    0/ 5994]
loss: 1.339026  [ 1600/ 5994]
loss: 1.309327  [ 3200/ 5994]
loss: 1.375575  [ 4800/ 5994]
Test Error: 
 Accuracy: 0.0%, Accuracy (Attribute-wise): 58.9%, Avg loss: 0.874802 

Epoch 3
-------------------------------
loss: 1.364762  [    0/ 5994]
loss: 1.256603  [ 1600/ 5994]
loss: 1.257102  [ 3200/ 5994]
loss: 1.293939  [ 4800/ 5994]
Test Error: 
 Accuracy: 0.0%, Accuracy (Attribute-wise): 61.7%, Avg loss: 0.806158 

Epoch 4
-------------------------------
loss: 1.284297  [    0/ 5994]
loss: 1.198305  [ 1600/ 5994]
loss: 1.187598  [ 3200/ 5994]
loss: 1.252539  [ 4800/ 5994]
Test Error: 
 Accuracy: 0.0%, Accuracy (Attribute-wise): 64.2%, Avg loss: 0.764079 

Epoch 5
------------